In [1]:
%matplotlib inline

import math
import time
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sb
import pandas as pd
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split, cross_val_score, learning_curve
from sklearn.neighbors import KNeighborsRegressor
from sklearn.datasets import load_iris, load_digits
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import WhiteKernel, ExpSineSquared
from sklearn.kernel_ridge import KernelRidge


In [2]:
def split_train_test(inputs, outputs):
    return train_test_split(inputs, outputs, test_size=0.25, shuffle=True)

def train_model(params, search, in_train, out_train):
    start = time.time()
    search.fit(in_train, out_train)
    end = time.time()
    elapsed_time = end - start
    print('Elapsed time: {}mins'.format(elapsed_time / 60))
    print('Best score: {}'.format(search.best_score_))
    print('Best parameters: {}'.format(search.best_params_))
    return search.best_estimator_

    #rf_estimator = RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                          #max_depth=100, max_features=3, max_leaf_nodes=None,
                          #max_samples=None, min_impurity_decrease=0.0,
                          #min_impurity_split=None, min_samples_leaf=3,
                          #min_samples_split=8, min_weight_fraction_leaf=0.0,
                          #n_estimators=100, n_jobs=None, oob_score=False,
                          #random_state=None, verbose=0, warm_start=False)
    #rf_estimator.fit(in_conf_train, out_conf_train)
    #return rf_estimator
    
def check_results_test(estimator, in_test, out_test, label):
    estimator_pred = estimator.predict(in_test)
    plt.plot(out_test)
    plt.plot(estimator_pred)
    plt.legend(['Input Data', label])
    print('MAE:', mean_absolute_error(out_test, estimator_pred))
    print('MSE:',mean_squared_error(out_test, estimator_pred))
    print('RMSE:', math.sqrt(mean_squared_error(out_test, estimator_pred)))
    print('R^2:', r2_score(out_test, estimator_pred))
    
def plot_cross_val_score(estimator, inputs, outputs, number_cv):
    cv_scores = cross_val_score(estimator, inputs, outputs, cv=number_cv)
    plt.hist(cv_scores)
    plt.title('Average score: {}'.format(np.mean(cv_scores)))
    
def calc_cumulative(in_list):
    out_list = []
    for idx, elem in enumerate(in_list):
        if idx == 0:
            out_list.append(elem)
        else:
            out_list.append(elem + out_list[idx - 1])
    return out_list
    
def plot_predictions_for_country(df, country, estimator, inputs_list, output, label_out, label_pred, title, cumulative=False):
    country_df = df.loc[df["Country/Region"] == country]
    country_inputs = country_df[inputs_list].values
    country_outputs = country_df[output].values
    days_list = df["Num_days_from_begin"].unique()
    country_pred_output = estimator.predict(country_inputs) 

    first_day = df["Date"].min()
    last_day = df["Date"].max()

    if cumulative:
        country_pred_output = calc_cumulative(country_pred_output)
    
    print('MAE:', mean_absolute_error(country_outputs, country_pred_output))
    print('MSE:',mean_squared_error(country_outputs, country_pred_output))
    print('RMSE:', math.sqrt(mean_squared_error(country_outputs, country_pred_output)))
    print('R^2:', r2_score(country_outputs, country_pred_output))

    plt.plot(days_list, country_outputs, color='blue', label=label_out)
    plt.plot(days_list, country_pred_output, color='green', label=label_pred)
    plt.xlabel("Number of days since {}".format(first_day))
    plt.ylabel("# of Cases")
    plt.title(title + "(as of {})".format(last_day))
    plt.legend()
    
def plot_learning_curve(estimator, inputs, outputs):
    train_sizes, train_scores, test_scores = learning_curve(estimator=estimator, X=inputs, y=outputs,
                                       groups=None, 
                                       train_sizes=np.array([0.1, 0.33, 0.55, 0.78, 1. ]),
                                       cv=None, scoring=None, exploit_incremental_learning=False, 
                                       n_jobs=None, pre_dispatch='all', verbose=0, shuffle=True,
                                       random_state=None, error_score=np.nan, return_times=False)

    train_scores_mean = train_scores.mean(axis=1)
    test_scores_mean = test_scores.mean(axis=1)

    plt.plot(train_sizes, train_scores_mean, label='Training score')
    plt.plot(train_sizes, test_scores_mean, label='Test score')

    plt.ylabel('Score', fontsize=14)
    plt.xlabel('Training set size', fontsize=14)
    title = 'Learning curves for a ' + str(estimator).split('(')[0] + ' model'
    plt.title(title, fontsize=18, y=1.03)
    plt.legend()
    
def plot_pred_diff(df, estimator, country, inputs_list, output):
    country_df = df.loc[df["Country/Region"] == country]
    country_inputs = country_df[inputs_list].values
    country_outputs = country_df[output].values
    days_list = df["Num_days_from_begin"].unique()
    country_pred_output = estimator.predict(country_inputs)

    first_day = df["Date"].min()
    last_day = df["Date"].max()

    diff_list = np.subtract(country_outputs, country_pred_output)
    plt.bar(days_list, diff_list, color='blue', label="")
    plt.show()

In [3]:
# load previously processed information into dataframe
df = pd.read_csv('data/covid_19_world_processed.csv', delimiter=',')
df

Country/Region      Lat  Lat_norm      Long  Long_norm        Date  \
0           Afghanistan  33.0000  0.471083   65.0000   0.597360  2020-01-22   
1               Albania  41.1533  0.798304   20.1683  -0.039787  2020-01-22   
2      Alberta (Canada)  53.9333  1.311212 -116.5765  -1.983201  2020-01-22   
3               Algeria  28.0339  0.271776    1.6596  -0.302833  2020-01-22   
4               Andorra  42.5063  0.852605    1.5218  -0.304791  2020-01-22   
...                 ...      ...       ...       ...        ...         ...   
30619    Yukon (Canada)  64.2823  1.726554 -135.0000  -2.245035  2020-05-16   
30620    Yunnan (China)  24.9740  0.148971  101.4870   1.115912  2020-05-16   
30621            Zambia -15.4167 -1.472053   28.2833   0.075543  2020-05-16   
30622  Zhejiang (China)  29.1832  0.317901  120.0934   1.380346  2020-05-16   
30623          Zimbabwe -20.0000 -1.655997   30.0000   0.099941  2020-05-16   

       Num_days_from_begin  Num_days_from_begin_norm  Confirmed  Deaths  ...  \
0                        0                 -1.717183          0       0  ...   
1                        0                 -1.717183          0       0  ...   
2                        0                 -1.717183          0       0  ...   
3                        0                 -1.717183          0       0  ...   
4                        0                 -1.717183          0       0  ...   
...                    ...                       ...        ...     ...  ...   
30619                  115                  1.717183         11       0  ...   
30620                  115                  1.717183        185       2  ...   
30621                  115                  1.717183        679       7  ...   
30622                  115                  1.717183       1268       1  ...   
30623                  115                  1.717183         42       4  ...   

       Confirmed_inc_day_before  Confirmed_inc_day_before_norm  Deaths_inc  \
0                           NaN                            NaN         0.0   
1                           NaN                            NaN         0.0   
2                           NaN                            NaN         0.0   
3                           NaN                            NaN         0.0   
4                           NaN                            NaN         0.0   
...                         ...                            ...         ...   
30619                       0.0                      -0.116999         0.0   
30620                       0.0                      -0.116999         0.0   
30621                       0.0                      -0.116999         0.0   
30622                       0.0                      -0.116999         0.0   
30623                       5.0                      -0.113089         0.0   

       Deaths_inc_norm  Deaths_inc_day_before  Deaths_inc_day_before_norm  \
0            -0.110666                    NaN                         NaN   
1            -0.110666                    NaN                         NaN   
2            -0.110666                    NaN                         NaN   
3            -0.110666                    NaN                         NaN   
4            -0.110666                    NaN                         NaN   
...                ...                    ...                         ...   
30619        -0.110666                    0.0                   -0.110177   
30620        -0.110666                    0.0                   -0.110177   
30621        -0.110666                    0.0                   -0.110177   
30622        -0.110666                    0.0                   -0.110177   
30623        -0.110666                    0.0                   -0.110177   

       Recovered_inc  Recovered_inc_norm  Recovered_inc_day_before  \
0                0.0           -0.121130                       NaN   
1                0.0           -0.121130                       NaN   
2                NaN       

In [4]:
# get the inputs of the data for the machine learning algorithm
inputs = df[["Lat_norm", "Long_norm", "Num_days_from_begin_norm"]].values
inputs

array([[ 0.47108303,  0.59735963, -1.71718314],
       [ 0.79830423, -0.03978737, -1.71718314],
       [ 1.3112115 , -1.98320106, -1.71718314],
       ...,
       [-1.47205293,  0.07554281,  1.71718314],
       [ 0.31790115,  1.38034566,  1.71718314],
       [-1.65599721,  0.09994051,  1.71718314]])

In [5]:
# get the output for the machine learning algorithm (confirmed inc cases)
outputs = df["Confirmed_inc"].values
outputs

array([ 0.,  0.,  0., ..., 25.,  0.,  0.])

In [6]:
# split the data into training and testing sets
in_conf_train, in_conf_test, out_conf_train, out_conf_test = split_train_test(inputs, outputs)

# NOTE: actually, the "testing" set will be used for "manual" testing and plotting, while the training set will be
# further divided in training and testing sets, in order to perform k-fold cross validation.

In [7]:
# create K-Nearest Neighbours instance, and check the hyper parameters)
kr = KernelRidge()
kr.get_params()

{'alpha': 1,
 'coef0': 1,
 'degree': 3,
 'gamma': None,
 'kernel': 'linear',
 'kernel_params': None}

In [ ]:
# do hyper parameter tuning with the K-Nearest, using cross validation
param_grid = {"alpha": [1e0, 1e-1, 1e-2, 1e-3],
              "kernel": [ExpSineSquared(l, p)
                         for l in np.logspace(-2, 2, 10)
                         for p in np.logspace(0, 2, 10)]}
search =RandomizedSearchCV(kr, param_grid, scoring='neg_mean_squared_error', cv=3, return_train_score=True, n_jobs=-1, n_iter=10, verbose=1)
kr_estimator = train_model(param_grid, search, in_conf_train, out_conf_train)
kr_estimator

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


In [ ]:
check_results_test(kr_estimator, in_conf_test, out_conf_test, 'Kernel Ridge Predictions')

In [ ]:
plot_predictions_for_country(
    df, "US", kr_estimator, 
    ["Lat_norm", "Long_norm", "Num_days_from_begin_norm"], 
    "Confirmed",
    "Actual Confirmed Cases", 
    "Predicted (Random Forest)", 
    "US Confirmed Cases vs Prediction from Nearest Neighbour",
    True
)

In [ ]:
gp_kernel = ExpSineSquared(1.0, 5.0, periodicity_bounds=(1e-2, 1e1)) \
    + WhiteKernel(1e-1)
gpr = GaussianProcessRegressor(kernel=gp_kernel)

In [ ]:
 gpr.fit(in_train, out_train)

In [ ]:
gpr_estimator = gpr.best_estimator_

In [ ]:
check_results_test(gpr_estimator, in_conf_test, out_conf_test, 'Kernel Ridge Predictions')

In [ ]:
plot_predictions_for_country(
    df, "US", gpr_estimator, 
    ["Lat_norm", "Long_norm", "Num_days_from_begin_norm"], 
    "Confirmed",
    "Actual Confirmed Cases", 
    "Predicted (Random Forest)", 
    "US Confirmed Cases vs Prediction from Nearest Neighbour",
    True
)